# データの整形

## モジュールのインポート

In [1]:
from polyglot.text import Text   #形態素解析

from nltk.tokenize import sent_tokenize

import numpy as np

import pandas as pd

import re  #正規表現

import glob

from natsort import natsorted

#プロット
%matplotlib inline
import matplotlib.pyplot as plt


from sklearn.preprocessing import LabelEncoder

## 多読本のデータの読み込み

In [2]:
def read_txt(filepath):
    f = open(filepath, 'r', encoding='UTF-8')
    data = f.read()
    f.close()
    return data

all_files = glob.glob('erfpt-data/ERFPT-planeData/' + '*.text')

# 自然順ソート
all_files = natsorted(all_files)


text = []
for file in all_files:
    text.append(read_txt(file))

In [3]:
df = pd.read_csv("./erfpt-data/erfpt-data.csv",  dtype=object)

df.insert(3, 'text', text)

df

,item_id,code,name,text,word_count,publisher,level
0,13,C2,Miranda,The children and their father remember Miranda...,518,Cambridge,5
1,14,P1,Girl Meets Boy,My story starts in late July. It was July 21st...,348,Penguin,6
2,15,O1,New York,One dollar and eighty seven cents. That was al...,629,Oxford,7
3,16,P2,Emil and the Detectives,"'Now, Emil,' said his mother, 'get ready your ...",529,Penguin,8
4,17,O3,Ethan Frome,"If you know Starkfield, Massachusetts, you kno...",699,Oxford,8
5,18,O6,Silas Marner,"In the early years of the nineteenth century, ...",647,Oxford,10
6,19,C5,High Life Low Life,It had been a long day. It was almost time for...,664,Cambridge,10
7,20,O7,Brat Farrar,"'Why can't you eat more politely, Jane? Like y...",666,Oxford,11
8,21,E4,[practice]Come Home,"OK, that's it for today' says Mr. Babic. 'Read...",374,Cengage,3
9,22,E2,The Beautiful Game,"""Yes!"" called Estela as she came off the field...",400,Cengage,4


## 文章を数値データへ

#### 形態素解析のときのメモ
- 空白と改行が入っていても正常に形態素解析が行われる

In [4]:
#クリーニング（テキスト中の記号除去）
def cleaning_text(text):
    code_regex = re.compile('[!"#$%&\'\\\\()*+,\/:;<=>?@[\\]^_`{|}~「」〔〕“”‘’〈〉『』【】＆＊・（）＄＃＠—–、？！｀＋￥％]')
    
    cleaned_text = code_regex.sub('', text)
    
    cleaned_text = cleaned_text.replace('-', ' ').replace('\'s', '')
    
#     ピリオド入れるか入れないか検討
    cleaned_text = cleaned_text.replace('.', '')
    
    return cleaned_text

df['text'] = df['text'].apply(cleaning_text)



#形態素解析
def morphologicalAnalysis(text):
    tokens = Text(text)
    
    tag_list = []
    for token in tokens.pos_tags:
            tag_list.append(token[1])
    return tag_list

df['maText'] = df['text'].apply(morphologicalAnalysis)




#ラベルエンコーディング
le = LabelEncoder()
tag_list = ['ADJ', 'ADP', 'ADV', 'AUX', 'CONJ', 'DET', 'INTJ', 'NOUN', 'NUM', 'PART', 'PRON', 'PROPN', 'PUNCT', 'SCONJ', 'VERB']
le.fit(tag_list)

def labelEncoding(maText):
    maText = list(filter(lambda element: element != 'X', maText))
    maText = list(filter(lambda element: element != 'SYM', maText))
    
    num_tag = le.transform(maText) 
    return num_tag

df['text NumData'] = df['maText'].apply(labelEncoding)

### ラベルエンコーディング タグ一覧

| No  |  TAG  |   意味       |  単語例   | 
| ---- | ------- |  ------------- |  :-------: | 
| 0   |  ADJ  |   形容詞     | big, old, green, incomprehensible, first | 
| 1   |  ADP  |   接置詞     | in, to, during | 
| 2   |  ADV  |   副詞       | very, well, exactly | 
| 3   |  AUX  |   助動詞     | is(doing), will(do), was(done), should(do) | 
| 4   | CONJ | 等位接続詞  | and, or, but | 
| 5   |  DET  |   限定詞      | the, a, an, this, that, my, your, a few, a little, one | 
| 6   |  INTJ |   間投詞      | ouch, bravo, hello, well, you know | 
| 7   | NOUN |  名詞         | girl, cat, tree, air, beauty | 
| 8   |  NUM  |  数詞         | 1998, ten, 12 | 
| 9   | PART |  不変化詞    | 's, not | 
| 10  | PRON |  代名詞      | I, you, he, it, they, myself, yourself, who, what | 
| 11  | PROPN |  固有名詞  | Mary, John, London, NATO, HBO | 
| 12  | PUNCT |  句読点     | .(ピリオド) | 
| 13  | SCONJ |  従属接続詞 | that, if, while | 
| 14  |  VERB |   動詞        | run. eat, runs. ate, running, eating | 


## スライド窓　カウント

In [5]:
%%time

ws = 3   #window size

for index, row in df.iterrows():
    
    array = row['text NumData']

    for i in range(len(array) - (ws - 1)): 

        window = array[i : i + ws]

        if (not '{}'.format(window) in df.columns):
            df['{}'.format(window)] = 0
        
        
        df.at[index, '{}'.format(window)] += 1

CPU times: user 5.1 s, sys: 32 ms, total: 5.13 s
Wall time: 5.06 s


In [6]:
df_num = df.iloc[:, 9:]
pd.set_option('display.max_columns', 1500)


# print(f'{df.shape[0]} rows ×　{df.shape[1]} columns')

## カウントを比率に正規化

In [7]:
def Normalization(num, word_count):
    ratio = num / (int(word_count)-2)
    return ratio
    
for index, row in df.iterrows():
    df_num[index : index + 1] = df_num[index : index + 1].apply( Normalization, axis=1, word_count=df.at[index, 'word_count'])

In [8]:
#百分率表示へ
df_num = df_num * 100

# display(df_num)

[5 7 4]  [ 7  4 10]  [ 4 10  7]  [10  7 14]  [ 7 14 11]  [14 11  1]  \
0   1.550388    1.550388    0.387597    0.775194    0.193798    0.387597   
1   1.445087    1.156069    1.156069    0.867052    0.000000    0.000000   
2   0.956938    0.956938    0.159490    0.478469    0.159490    0.000000   
3   0.569260    0.948767    0.948767    2.087287    0.189753    0.000000   
4   0.860832    0.573888    0.286944    0.286944    0.286944    0.000000   
5   0.465116    0.310078    0.310078    0.155039    0.155039    0.000000   
6   0.151057    0.151057    0.000000    0.151057    0.000000    0.000000   
7   0.150602    0.602410    0.451807    0.301205    0.301205    0.150602   
8   0.268817    0.268817    0.000000    0.268817    0.000000    0.000000   
9   0.251256    0.502513    0.502513    0.502513    0.000000    0.000000   
10  0.516796    0.258398    0.258398    0.258398    0.000000    0.000000   
11  0.320513    0.320513    0.320513    0.320513    0.320513    0.000000   
12  1.012146    0.404858    0.202429    0.607287    0.000000    0.000000   
13  0.195312    0.390625    0.390625    0.585938    0.195312    0.195312   
14  0.000000    0.199203    0.597610    2.390438    0.199203    0.000000   
15  0.396825    0.198413    0.198413    0.396825    0.000000    0.000000   
16  1.818182    0.606061    0.000000    0.000000    0.000000    0.000000   
17  0.546448    0.546448    0.182149    0.546448    0.182149    0.000000   
18  0.779221    0.259740    0.000000    0.259740    0.259740    0.000000   
19  0.346620    0.346620    0.000000    0.173310    0.000000    0.173310   
20  0.000000    0.000000    0.000000    0.355240    0.710480    0.000000   
21  0.000000    0.000000    0.000000    1.156069    1.734104    0.578035   
22  0.790514    0.395257    0.197628    0.197628    0.395257    0.000000   
23  0.459770    0.229885    0.000000    0.689655    0.919540    0.000000   
24  0.000000    0.000000    0.000000    0.405680    0.202840    0.000000   
25  0.399202    0.199601    0.399202    0.598802    0.000000    0.000000   
26  0.413223    0.206612    0.000000    0.826446    0.206612    0.000000   
27  0.584795    0.292398    0.584795    0.584795    0.000000    0.000000   
28  0.664452    0.166113    0.166113    0.498339    1.495017    0.166113   
29  0.198020    0.198020    0.000000    0.396040    0.594059    0.000000   
30  0.398406    0.199203    0.000000    0.000000    0.796813    0.000000   
31  0.198020    0.594059    0.000000    0.000000    0.594059    0.000000   
32  0.182815    0.000000    0.000000    0.182815    1.096892    0.000000   
33  0.417101    0.104275    0.000000    0.521376    0.000000    0.000000   
34  0.256082    0.000000    0.128041    0.640205    0.000000    0.000000   
35  0.717360    0.000000    0.000000    0.286944    0.143472    0.143472   

    [11  1  7]   [1 7 4]  [ 4 10 14]  [10 14  1]  [14  1 10]  [ 1 10  2]  \
0     0.193798  0.193798    1.550388    2.325581    0.581395    0.193798   
1     0.000000  0.289017    1.445087    1.734104    1.445087    0.289017   
2     0.000000  0.000000    1.594896    1.754386    1.275917    0.159490   
3     0.000000  0.189753    0.189753    0.569260    1.518027    0.379507   
4     0.000000  0.000000    0.286944    0.717360    0.717360    0.000000   
5     0.000000  0.000000    0.000000    0.465116    0.310078    0.000000   
6     0.000000  0.151057    0.000000    0.604230    0.604230    0.302115   
7     0.000000  0.000000    0.301205    0.753012    1.204819    0.150602   
8     0.000000  0.268817    1.612903    2.688172    1.344086    0.000000   
9     0.000000  0.251256    0.251256    1.005025    0.502513    0.000000   
10    0.000000  0.258398    0.516796    1.550388    0.516796    0.000000   
11    0.000000  0.000000    1.282051    0.320513    0.641026    0.320513   
12    0.000000  0.000000    0.607287    0.809717    0.404858    0.000000   
13    0.000000  0.000000    0.390625    1.562500    0.781250    0.000000   
14    0.000000  0.000000    0.000000    0.597610    

In [10]:
#抽出する標準偏差の値
#stDv以上の標準偏差列を抽出する
stDv = 0.2


df_Num = df_num.loc[:, df_num.std() > stDv]

# display(df_Num)

print(f'{df_Num.shape[0]} rows ×　{df_Num.shape[1]} columns')

,[5 7 4],[ 7 4 10],[ 4 10 7],[10 7 14],[ 7 14 11],[ 4 10 14],[10 14 1],[14 1 10],[ 7 2 14],[ 2 14 10],[10 4 10],[10 2 14],[14 5 7],[5 7 5],[7 5 7],[5 7 1],[ 7 1 10],[ 1 10 7],[10 7 11],[ 7 11 14],[11 14 11],[14 11 10],[11 10 14],[10 14 10],[14 10 3],[1 5 7],[5 7 7],[ 7 7 14],[ 7 14 2],[14 2 10],[ 2 14 11],[14 11 4],[4 5 7],[ 5 7 10],[ 7 10 14],[10 7 2],[2 1 5],[7 1 5],[ 1 10 0],[0 0 7],[0 7 4],[2 0 7],[ 0 7 10],[ 7 2 10],[ 2 10 14],[10 14 5],[14 5 0],[10 14 11],[10 0 7],[0 7 1],[ 7 1 11],[ 1 11 10],[10 14 2],[14 2 1],[14 10 14],[10 14 0],[14 0 4],[ 5 14 5],[5 0 7],[10 14 9],[14 9 14],[ 9 14 5],[13 10 14],[14 1 11],[14 1 5],[10 7 10],[14 10 2],[2 5 7],[ 5 7 14],[ 7 14 1],[ 1 10 4],[ 3 10 14],[14 2 2],[ 2 2 10],[14 11 11],[11 11 11],[ 7 14 10],[ 4 10 3],[10 3 14],[ 3 14 2],[ 2 14 1],[11 11 4],[ 7 10 3],[ 2 1 10],[10 7 4],[ 1 11 11],[5 7 2],[ 9 14 1],[ 7 4 14],[ 4 14 1],[ 7 10 7],[14 7 1],[ 2 2 14],[7 4 5],[14 10 10],[10 10 14],[5 7 3],[14 10 7],[ 7 14 0],[0 4 0],[1 5 0],[5 0 0],[0 7 5],[ 0 10 14],[ 4 11 14],[11 14 5],[7 4 7],[ 5 14 10],[ 9 14 0],[14 0 1],[0 1 5],[ 7 14 5],[ 2 0 10],[7 5 0],[8 1 5],[ 1 11 1],[ 7 11 11],[11 11 14],[11 14 1],[10 1 5],[14 2 0],[2 0 1],[ 0 14 11],[ 3 14 10],[14 10 1],[7 1 7],[10 7 7],[ 7 7 10],[10 7 1],[ 7 9 14],[ 0 7 14],[14 11 14],[ 0 7 11],[11 14 0],[ 0 1 10],[11 14 2],[11 14 10],[14 0 7],[ 9 14 10],[ 2 11 14],[ 1 5 11],[8 7 7],[10 3 9],[ 3 9 14],[10 14 7],[13 10 3],[ 3 14 1],[7 7 1],[ 5 7 11],[11 10 3],[1 5 8],[11 14 9],[ 2 10 3],[ 3 14 9],[14 13 10],[ 7 3 14],[1 0 7],[14 1 7],[ 1 7 10],[5 8 7],[11 3 14],[ 3 14 5],[ 1 11 14],[11 1 5],[14 14 11],[ 7 11 4],[11 4 11],[ 5 11 14],[5 2 0],[ 7 5 11]
0,1.550388,1.550388,0.387597,0.775194,0.193798,1.550388,2.325581,0.581395,0.193798,0.193798,0.193798,0.581395,1.356589,0.581395,1.162791,2.519380,0.775194,1.356589,0.193798,0.193798,0.193798,0.387597,0.968992,1.550388,0.193798,4.844961,0.581395,0.193798,0.193798,0.193798,0.193798,0.193798,0.775194,2.131783,2.325581,0.387597,0.387597,1.937984,0.387597,0.387597,0.581395,0.193798,0.193798,0.581395,0.968992,1.356589,0.968992,0.581395,0.387597,0.387597,0.387597,0.387597,0.968992,0.581395,0.581395,0.775194,0.775194,0.581395,0.968992,0.581395,0.775194,0.193798,0.387597,0.387597,1.937984,0.193798,0.193798,0.193798,1.356589,0.581395,0.193798,0.581395,0.581395,0.193798,0.387597,0.193798,0.387597,0.193798,0.387597,0.581395,0.387597,0.193798,0.193798,0.193798,0.581395,0.387597,0.387597,0.387597,0.193798,0.193798,0.193798,0.193798,0.193798,0.193798,0.387597,0.387597,0.193798,0.193798,0.387597,0.581395,0.193798,0.193798,0.193798,0.387597,0.193798,0.193798,0.193798,0.193798,0.193798,0.193798,0.193798,0.193798,0.193798,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,1.445087,1.156069,1.156069,0.867052,0.000000,1.445087,1.734104,1.445087,0.000000,0.000000,1.734104,0.000000,1.445087,0.867052,1.156069,2.312139,2.312139,1.156069,0.289017,0.000000,0.000000,0.000000,0.578035,1.445087,0.000000,4.046243,0.578035,0.000000,0.000000,0.289017,0.000000,0.000000,0.867052,1.445087,2.312139,0.289017,0.578035,1.156069,0.578035,0.578035,0.578035,0.000000,0.867052,0.000000,1.156069,1.156069,1.156069,0.289017,0.867052,2.023121,0.867052,0.289017,0.867052,0.289017,0.867052,0.289017,0.289017,0.578035,2.312139,0.578035,0.867052,0.289017,0.000000,0.000000,2.023121,0.578035,0.289017,0.289017,0.867052,0.578035,1.156069,0.000000,0.289017,0.578035,0.000000,0.289017,0.578035,0.000000,0.289017,0.000000,0.000000,0.000000,0.000000,0.000000,0.867052,0.000000,0.578035,0.289017,0.867052,0.578035,0.578

36 rows ×　170 columns


In [17]:
shaped_df = pd.concat([df.loc[:, "item_id" : "level"], df_Num], axis=1)
print(f'{shaped_df.shape[0]} rows ×　{shaped_df.shape[1]} columns')

shaped_df

36 rows ×　177 columns


,item_id,code,name,text,word_count,publisher,level,[5 7 4],[ 7 4 10],[ 4 10 7],[10 7 14],[ 7 14 11],[ 4 10 14],[10 14 1],[14 1 10],[ 7 2 14],[ 2 14 10],[10 4 10],[10 2 14],[14 5 7],[5 7 5],[7 5 7],[5 7 1],[ 7 1 10],[ 1 10 7],[10 7 11],[ 7 11 14],[11 14 11],[14 11 10],[11 10 14],[10 14 10],[14 10 3],[1 5 7],[5 7 7],[ 7 7 14],[ 7 14 2],[14 2 10],[ 2 14 11],[14 11 4],[4 5 7],[ 5 7 10],[ 7 10 14],[10 7 2],[2 1 5],[7 1 5],[ 1 10 0],[0 0 7],[0 7 4],[2 0 7],[ 0 7 10],[ 7 2 10],[ 2 10 14],[10 14 5],[14 5 0],[10 14 11],[10 0 7],[0 7 1],[ 7 1 11],[ 1 11 10],[10 14 2],[14 2 1],[14 10 14],[10 14 0],[14 0 4],[ 5 14 5],[5 0 7],[10 14 9],[14 9 14],[ 9 14 5],[13 10 14],[14 1 11],[14 1 5],[10 7 10],[14 10 2],[2 5 7],[ 5 7 14],[ 7 14 1],[ 1 10 4],[ 3 10 14],[14 2 2],[ 2 2 10],[14 11 11],[11 11 11],[ 7 14 10],[ 4 10 3],[10 3 14],[ 3 14 2],[ 2 14 1],[11 11 4],[ 7 10 3],[ 2 1 10],[10 7 4],[ 1 11 11],[5 7 2],[ 9 14 1],[ 7 4 14],[ 4 14 1],[ 7 10 7],[14 7 1],[ 2 2 14],[7 4 5],[14 10 10],[10 10 14],[5 7 3],[14 10 7],[ 7 14 0],[0 4 0],[1 5 0],[5 0 0],[0 7 5],[ 0 10 14],[ 4 11 14],[11 14 5],[7 4 7],[ 5 14 10],[ 9 14 0],[14 0 1],[0 1 5],[ 7 14 5],[ 2 0 10],[7 5 0],[8 1 5],[ 1 11 1],[ 7 11 11],[11 11 14],[11 14 1],[10 1 5],[14 2 0],[2 0 1],[ 0 14 11],[ 3 14 10],[14 10 1],[7 1 7],[10 7 7],[ 7 7 10],[10 7 1],[ 7 9 14],[ 0 7 14],[14 11 14],[ 0 7 11],[11 14 0],[ 0 1 10],[11 14 2],[11 14 10],[14 0 7],[ 9 14 10],[ 2 11 14],[ 1 5 11],[8 7 7],[10 3 9],[ 3 9 14],[10 14 7],[13 10 3],[ 3 14 1],[7 7 1],[ 5 7 11],[11 10 3],[1 5 8],[11 14 9],[ 2 10 3],[ 3 14 9],[14 13 10],[ 7 3 14],[1 0 7],[14 1 7],[ 1 7 10],[5 8 7],[11 3 14],[ 3 14 5],[ 1 11 14],[11 1 5],[14 14 11],[ 7 11 4],[11 4 11],[ 5 11 14],[5 2 0],[ 7 5 11]
0,13,C2,Miranda,The children and their father remember Miranda...,518,Cambridge,5,1.550388,1.550388,0.387597,0.775194,0.193798,1.550388,2.325581,0.581395,0.193798,0.193798,0.193798,0.581395,1.356589,0.581395,1.162791,2.519380,0.775194,1.356589,0.193798,0.193798,0.193798,0.387597,0.968992,1.550388,0.193798,4.844961,0.581395,0.193798,0.193798,0.193798,0.193798,0.193798,0.775194,2.131783,2.325581,0.387597,0.387597,1.937984,0.387597,0.387597,0.581395,0.193798,0.193798,0.581395,0.968992,1.356589,0.968992,0.581395,0.387597,0.387597,0.387597,0.387597,0.968992,0.581395,0.581395,0.775194,0.775194,0.581395,0.968992,0.581395,0.775194,0.193798,0.387597,0.387597,1.937984,0.193798,0.193798,0.193798,1.356589,0.581395,0.193798,0.581395,0.581395,0.193798,0.387597,0.193798,0.387597,0.193798,0.387597,0.581395,0.387597,0.193798,0.193798,0.193798,0.581395,0.387597,0.387597,0.387597,0.193798,0.193798,0.193798,0.193798,0.193798,0.193798,0.387597,0.387597,0.193798,0.193798,0.387597,0.581395,0.193798,0.193798,0.193798,0.387597,0.193798,0.193798,0.193798,0.193798,0.193798,0.193798,0.193798,0.193798,0.193798,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,14,P1,Girl Meets Boy,My story starts in late July It was July 21st ...,348,Penguin,6,1.445087,1.156069,1.156069,0.867052,0.000000,1.445087,1.734104,1.445087,0.000000,0.000000,1.734104,0.000000,1.445087,0.867052,1.156069,2.312139,2.312139,1.156069,0.289017,0.000000,0.000000,0.000000,0.578035,1.445087,0.000000,4.046243,0.578035,0.000000,0.000000,0.289017,0.000000,0.000000,0.867052,1.445087,2.312139,0.289017,0.578035,1.156069,0.578035,0.578035,0.578035,0.000000,0.867052,0.000000,1.156069,1.156069,1.156069,0.289017,0.867052,2.023121,0.867052,0.289017,0.867052,0.289017,0.867052,0.289017,0.289017,0.578035,2.312139,0.578035,0.867052,0.289017,0.000000,0.000000,2.023121,0.578035,0.2890

In [18]:
#データ保存

file_name = 'dataframe'
shaped_df.to_csv('./ER-numData/' + file_name + '.csv')

# 機械学習

In [19]:
df = pd.read_csv("./ER-numData/dataframe.csv",  index_col=0, dtype=object)

df

,item_id,code,name,text,word_count,publisher,level,[5 7 4],[ 7 4 10],[ 4 10 7],[10 7 14],[ 7 14 11],[ 4 10 14],[10 14 1],[14 1 10],[ 7 2 14],[ 2 14 10],[10 4 10],[10 2 14],[14 5 7],[5 7 5],[7 5 7],[5 7 1],[ 7 1 10],[ 1 10 7],[10 7 11],[ 7 11 14],[11 14 11],[14 11 10],[11 10 14],[10 14 10],[14 10 3],[1 5 7],[5 7 7],[ 7 7 14],[ 7 14 2],[14 2 10],[ 2 14 11],[14 11 4],[4 5 7],[ 5 7 10],[ 7 10 14],[10 7 2],[2 1 5],[7 1 5],[ 1 10 0],[0 0 7],[0 7 4],[2 0 7],[ 0 7 10],[ 7 2 10],[ 2 10 14],[10 14 5],[14 5 0],[10 14 11],[10 0 7],[0 7 1],[ 7 1 11],[ 1 11 10],[10 14 2],[14 2 1],[14 10 14],[10 14 0],[14 0 4],[ 5 14 5],[5 0 7],[10 14 9],[14 9 14],[ 9 14 5],[13 10 14],[14 1 11],[14 1 5],[10 7 10],[14 10 2],[2 5 7],[ 5 7 14],[ 7 14 1],[ 1 10 4],[ 3 10 14],[14 2 2],[ 2 2 10],[14 11 11],[11 11 11],[ 7 14 10],[ 4 10 3],[10 3 14],[ 3 14 2],[ 2 14 1],[11 11 4],[ 7 10 3],[ 2 1 10],[10 7 4],[ 1 11 11],[5 7 2],[ 9 14 1],[ 7 4 14],[ 4 14 1],[ 7 10 7],[14 7 1],[ 2 2 14],[7 4 5],[14 10 10],[10 10 14],[5 7 3],[14 10 7],[ 7 14 0],[0 4 0],[1 5 0],[5 0 0],[0 7 5],[ 0 10 14],[ 4 11 14],[11 14 5],[7 4 7],[ 5 14 10],[ 9 14 0],[14 0 1],[0 1 5],[ 7 14 5],[ 2 0 10],[7 5 0],[8 1 5],[ 1 11 1],[ 7 11 11],[11 11 14],[11 14 1],[10 1 5],[14 2 0],[2 0 1],[ 0 14 11],[ 3 14 10],[14 10 1],[7 1 7],[10 7 7],[ 7 7 10],[10 7 1],[ 7 9 14],[ 0 7 14],[14 11 14],[ 0 7 11],[11 14 0],[ 0 1 10],[11 14 2],[11 14 10],[14 0 7],[ 9 14 10],[ 2 11 14],[ 1 5 11],[8 7 7],[10 3 9],[ 3 9 14],[10 14 7],[13 10 3],[ 3 14 1],[7 7 1],[ 5 7 11],[11 10 3],[1 5 8],[11 14 9],[ 2 10 3],[ 3 14 9],[14 13 10],[ 7 3 14],[1 0 7],[14 1 7],[ 1 7 10],[5 8 7],[11 3 14],[ 3 14 5],[ 1 11 14],[11 1 5],[14 14 11],[ 7 11 4],[11 4 11],[ 5 11 14],[5 2 0],[ 7 5 11]
0,13,C2,Miranda,The children and their father remember Miranda...,518,Cambridge,5,1.550387596899225,1.550387596899225,0.3875968992248062,0.7751937984496124,0.1937984496124031,1.550387596899225,2.3255813953488373,0.5813953488372093,0.1937984496124031,0.1937984496124031,0.1937984496124031,0.5813953488372093,1.3565891472868217,0.5813953488372093,1.1627906976744187,2.5193798449612403,0.7751937984496124,1.3565891472868217,0.1937984496124031,0.1937984496124031,0.1937984496124031,0.3875968992248062,0.9689922480620154,1.550387596899225,0.1937984496124031,4.844961240310078,0.5813953488372093,0.1937984496124031,0.1937984496124031,0.1937984496124031,0.1937984496124031,0.1937984496124031,0.7751937984496124,2.131782945736434,2.3255813953488373,0.3875968992248062,0.3875968992248062,1.937984496124031,0.3875968992248062,0.3875968992248062,0.5813953488372093,0.1937984496124031,0.1937984496124031,0.5813953488372093,0.9689922480620154,1.3565891472868217,0.9689922480620154,0.5813953488372093,0.3875968992248062,0.3875968992248062,0.3875968992248062,0.3875968992248062,0.9689922480620154,0.5813953488372093,0.5813953488372093,0.7751937984496124,0.7751937984496124,0.5813953488372093,0.9689922480620154,0.5813953488372093,0.7751937984496124,0.1937984496124031,0.3875968992248062,0.3875968992248062,1.937984496124031,0.1937984496124031,0.1937984496124031,0.1937984496124031,1.3565891472868217,0.5813953488372093,0.1937984496124031,0.5813953488372093,0.5813953488372093,0.1937984496124031,0.3875968992248062,0.1937984496124031,0.3875968992248062,0.1937984496124031,0.3875968992248062,0.5813953488372093,0.3875968992248062,0.1937984496124031,0.1937984496124031,0.1937984496124031,0.5813953488372093,0.3875968992248062,0.3875968992248062,0.3875968992248062,0.1937984496124031,0.1937984496124031,0.1937984496124031,0.1937984496124031,0.1937984496124031,0.1937984496124031,0.3875968992248062,0.3875968992248062,0.1937984496124031,0.1937984496124031,0.3875968992248062,0.5813953488372093,0.1937984496124031,0.1937984496124031,0.1937984496124031,0.3875968992248062,0.1937984496124031,0.1937984496124031,0.1937984496124031,0.1937984496124031,0.1937984496124031,0.1937984496124031,0.1937984496124031,0.1937984496124031,0.1937984496124031,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.